# Indicador 4: Oportunidad de Notifiación 

## Objetivo: Mejorar la oportunidad de notificación de casos confirmados por laboratorio

### Meta: 100% 

**Periodicidad:** Semanal

**Escala territorial:** Comunal, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador


**Cálculo del indicador a nivel comunal:** 

$\frac{No.\:Casos\:confirmados\:notificados\:correctamente\:por\:comuna\:de\:residencia\:semana\:x}{No\:casos\:confirmados\:notificados\:por\:comuna\:residencia\:semana\:x} $

**Cálculo del indicador a nivel regional**

$\frac{No.\:Casos\:confirmados\:notificados\:correctamente\:por\:región\:de\:residencia\:semana\:x}{No\:casos\:confirmados\:notificados\:por\:región\:residencia\:semana\:x}$

**Observación**

- Se consideran sólo registros desde notificación
- Para los registros de notificaciones que se encuentren en gestión BAC, se considera la fecha de notificación BAC si esta es anterior a la fecha de notificación de EPIVIGLIA

## Importación de datos y librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa base de datos generada mediante merging 

data = pd.read_csv("data/data_con_establecimiento_cp.csv", 
                   low_memory=False
                  )

In [3]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = data[data['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,Unnamed: 0,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,...,fecha_resultado_otro_cultivo_3,factor_riesgo_caso_confirmado,factor_riesgo_caso_sospechoso,factor_riesgo_viaje_internacional,factor_riesgo_viaje_nacional,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi
0,0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coyhaique,Coyhaique


## Variables de importancia

|Variable|Tipo|Descripción|Observación|
|---|---|---|---|
|`region_residencia`| Text | Residencia registrada | Variable utilizada pero no mostrada
|`numero_folio`|Numeric| id | el número de folio se mantendrá por trazabilidad
|`comuna_residencia`|Text| City | Comuna de residencia informada
|`tipo_identificacion`|Text| Tipo de documento de identidad | Carnet o Pasaporte
|`identificacion_paciente`|Numeric|Rol Unico Nominal o Numero de pasaporte| |
|`fecha_notificacion`|DateTime| Fecha de notificación| DateTime en formato `%Y-%m-%d` |
|`semana_epidemiologica`|Numeric|Semana epidemiologica|valores de 1 al 52|
|`etapa_clinica`|Text|`SOSPECHA` o `BUSQUEDA A`| Debe ser filtrada para evitar valores `DESCARTADA` |
|`resultado_pcr_1`|Text||
|`fecha_resultado_pcr_1`|DateTime||
|`fecha_toma_muestra_1`|DateTime||
|`resultado_pcr_2`|Text||
|`fecha_resultado_pcr_2`|DateTime||
|`fecha_toma_muestra_2`|DateTime||
|`resultado_pcr_3`|Text||
|`fecha_resultado_pcr_3`|DateTime||
|`fecha_toma_muestra_3`|DateTime||
|`establecimiento_salud`|Text|Establecimiento de salud que reporta caso||
|`comuna_seremi_xi`|Text|Comuna identificada a traves de establecimiento_salud||
|`provincia_seremi_xi`|Text|Comuna identificada a traves de establecimiento_salud||


In [4]:
# Selecciona variables de importancia


data_aysen_clean =  data_aysen[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'establecimiento_salud',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi'
                               ]]


In [5]:
# Muestra valores en la variable etapa_clinica y cuenta preponderancia

data_aysen_clean['etapa_clinica'].value_counts()

DESCARTADA    135708
CONFIRMADA      8273
BUSQUEDA A      2468
PROBABLE        1533
SOSPECHA         805
Name: etapa_clinica, dtype: int64

In [ ]:
# Filtra etapa_clinica SOSPECHA

data_aysen_clean = data_aysen_clean[(b=='BUSQUEDA A')&(data_aysen_clean['etapa_clinica']=='CONFIRMADA')]